# Classificação de Diabetes com KNN

Este notebook estrutura o desenvolvimento de um modelo de classificação utilizando o algoritmo K-Nearest Neighbors (KNN) para a previsão da presença de diabetes, com base no conjunto de dados Pima Indians Diabetes. O trabalho segue um fluxo completo de projeto em aprendizado de máquina, cobrindo desde a preparação dos dados até a avaliação crítica do desempenho do modelo, com ênfase em rigor metodológico e reprodutibilidade dos resultados.

### 1. Instalação e Importação de Bibliotecas

In [ ]:
# Instalação das bibliotecas necessárias
# %%capture evita que as saídas da instalação poluam o notebook, mantendo-o limpo.
%%capture
%pip install numpy pandas matplotlib seaborn scikit-learn

In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Scikit-learn para modelagem e pré-processamento
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Modelos adicionais para comparação (importados apenas se forem de fato utilizados no projeto)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Configurações globais para visualização (opcional, mas recomendado para padronizar a estética dos gráficos)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6) # Tamanho padrão para figuras
plt.rcParams['axes.titlesize'] = 16    # Tamanho da fonte do título dos eixos
plt.rcParams['axes.labelsize'] = 12    # Tamanho da fonte dos rótulos dos eixos
plt.rcParams['xtick.labelsize'] = 10   # Tamanho da fonte dos ticks do eixo X
plt.rcParams['ytick.labelsize'] = 10   # Tamanho da fonte dos ticks do eixo Y

### 2. Carregamento e Visão Geral dos Dados

In [ ]:
# Carrega o dataset a partir de um arquivo CSV armazenado localmente.
df = pd.read_csv('data/diabetes.csv')

# Exibe as cinco primeiras entradas da base para verificação da estrutura e conteúdo dos dados.
print("Primeiras 5 linhas do dataset:")
print(df.head())

# Exibe a dimensão da base de dados (número de registros e atributos disponíveis).
print("\nFormato do dataset (linhas, colunas):", df.shape)

# Mostra os tipos de dados, nomes das colunas e presença de valores ausentes.
print("\nInformações sobre as colunas e tipos de dados:")
df.info()

# Resume estatísticas básicas das variáveis numéricas (média, desvio, quartis etc.).
print("\nEstatísticas descritivas das variáveis numéricas:")
print(df.describe())

# Verifica a distribuição da variável alvo, útil para identificar desbalanceamento.
print("\nContagem de valores na variável alvo 'Outcome':")
print(df['Outcome'].value_counts())

# Etapa exploratória essencial para conhecer a estrutura e equilíbrio dos dados.

### 3. Pré-processamento de Dados

In [ ]:
# Define as colunas onde o valor zero representa ausência de dado e não um valor válido.
columns_with_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Substitui zeros por NaN para permitir tratamento adequado de dados ausentes.
df[columns_with_zeros] = df[columns_with_zeros].replace(0, np.nan)

# Verifica quantos valores ausentes existem após a substituição dos zeros.
print("Valores nulos após substituição de 0 por NaN:")
print(df.isnull().sum())

# Preenche valores ausentes com a média nas colunas de distribuição simétrica.
for col in ['Glucose', 'BloodPressure', 'BMI']:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].mean())
        print(f"Valores ausentes em '{col}' preenchidos com a média.")

# Preenche com a mediana nas colunas com assimetria ou presença de outliers.
for col in ['SkinThickness', 'Insulin']:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())
        print(f"Valores ausentes em '{col}' preenchidos com a mediana.")

# Verifica se todas as imputações foram concluídas corretamente.
print("\nContagem de valores nulos após a imputação final:")
print(df[columns_with_zeros].isnull().sum())

### 4. Análise Exploratória de Dados (EDA) Aprofundada

In [ ]:
# Gera histogramas para todas as variáveis numéricas do dataset, permitindo observar a distribuição, presença de assimetrias e possíveis outliers.
df.hist(figsize=(15, 10), bins=10)

# Define um título geral para a grade de subplots gerada.
plt.suptitle('Histogramas das Variáveis do Dataset', y=1.02)

# Ajusta o layout para que o título e os gráficos não se sobreponham.
plt.tight_layout(rect=[0, 0.03, 1, 0.98])

# Exibe os gráficos.
plt.show()

In [ ]:
# Define o tamanho da figura para facilitar a leitura do mapa de calor.
plt.figure(figsize=(10, 8))

# Gera a matriz de correlação com anotação dos valores e esquema de cores.
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f")

# Adiciona um título ao gráfico para contextualizar a visualização.
plt.title('Matriz de Correlação das Variáveis')

# Exibe o gráfico na tela.
plt.show()

In [ ]:
# Define as variáveis numéricas que serão analisadas por diagnóstico.
numeric_features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
                    'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

# Cria a figura com tamanho apropriado para os subplots.
plt.figure(figsize=(15, 12))

# Itera sobre cada variável para gerar os box plots segmentados por Outcome.
for i, feature in enumerate(numeric_features):
    # Define a posição do subplot na grade (3 linhas x 3 colunas).
    ax = plt.subplot(3, 3, i + 1)

    # Cria o box plot da variável atual comparando grupos com e sem diabetes.
    sns.boxplot(x='Outcome', y=feature, data=df, palette='viridis', ax=ax, hue='Outcome', legend=False)

    # Adiciona o título individual para cada gráfico.
    plt.title(f'Distribuição de {feature} por Outcome')

    # Define o rótulo do eixo X com descrição textual.
    plt.xlabel('Diagnóstico de Diabetes')

    # Define o rótulo do eixo Y com o nome da variável analisada.
    plt.ylabel(feature)

    # Substitui os valores binários por rótulos interpretáveis no eixo X.
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['Não Diabético', 'Diabético'])

# Ajusta o layout da figura para evitar sobreposições.
plt.tight_layout()

# Exibe todos os box plots.
plt.show()

In [ ]:
# Cria nova figura para visualização da densidade das variáveis por grupo.
plt.figure(figsize=(15, 12))

# Itera sobre as mesmas variáveis para gerar os violin plots.
for i, feature in enumerate(numeric_features):
    # Define a posição do subplot na grade.
    ax = plt.subplot(3, 3, i + 1)

    # Gera o violin plot que combina distribuição com medidas de tendência central.
    sns.violinplot(x='Outcome', y=feature, data=df, palette='magma', ax=ax, hue='Outcome', legend=False)

    # Título do gráfico com a variável analisada.
    plt.title(f'Densidade de {feature} por Outcome')

    # Define rótulo explicativo do eixo X.
    plt.xlabel('Diagnóstico de Diabetes')

    # Define o eixo Y com o nome da variável.
    plt.ylabel(feature)

    # Aplica rótulos descritivos para os grupos.
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['Não Diabético', 'Diabético'])

# Ajusta os elementos da figura.
plt.tight_layout()

# Exibe todos os violin plots.
plt.show()


### 5. Preparação dos Dados para o Modelo

In [ ]:
# Separa as variáveis explicativas (features) removendo a variável alvo 'Outcome'.
X = df.drop('Outcome', axis=1)

# Define 'Outcome' como variável alvo, que indica presença (1) ou ausência (0) de diabetes.
y = df['Outcome']

# Instancia o objeto StandardScaler para normalização das variáveis.
scaler = StandardScaler()

# Aplica a padronização nas features, ajustando média para 0 e desvio padrão para 1.
# Essa etapa é essencial para algoritmos sensíveis à escala, como o KNN.
X_scaled = scaler.fit_transform(X)

# (Opcional) Conversão para DataFrame para visualização dos dados normalizados.
# print("\nPrimeiras 5 linhas dos dados escalonados:")
# print(X_scaled_df.head())

# Separa os dados em treino (70%) e teste (30%), preservando a proporção das classes com stratify.
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.30, random_state=101, stratify=y
)

# Exibe o formato do conjunto de treino após a divisão.
print(f"\nShape do conjunto de treino: {X_train_scaled.shape}")

# Exibe o formato do conjunto de teste após a divisão.
print(f"Shape do conjunto de teste: {X_test_scaled.shape}")

### 6. Treinamento e Otimização do Modelo KNN

In [ ]:
# Define a grade de valores de k (n_neighbors) a serem testados no modelo.
param_grid = {'n_neighbors': np.arange(1, 26)}  # Testa k de 1 a 25.

# Instancia o classificador KNN sem configurar k ainda.
knn = KNeighborsClassifier()

# Configura o GridSearchCV para buscar o melhor valor de k usando 5-fold cross-validation.
# scoring='accuracy' define que a métrica de avaliação será acurácia.
# n_jobs=-1 ativa o uso de todos os núcleos da CPU (execução paralela).
knn_cv = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Inicia o processo de busca pelos melhores hiperparâmetros no conjunto de treino escalado.
print("Iniciando busca de hiperparâmetros com GridSearchCV...")
knn_cv.fit(X_train_scaled, y_train)
print("Busca de hiperparâmetros concluída.")

# Exibe os melhores hiperparâmetros encontrados após a validação cruzada.
print(f"\nMelhores parâmetros encontrados: {knn_cv.best_params_}")
print(f"Melhor acurácia (média da validação cruzada no treino): {knn_cv.best_score_:.4f}")

# Extrai o melhor modelo treinado com o valor ideal de k encontrado.
best_knn_model = knn_cv.best_estimator_

# Converte os resultados detalhados da validação cruzada em DataFrame para visualização.
results = pd.DataFrame(knn_cv.cv_results_)

# Cria gráfico de linha para visualizar o desempenho médio por valor de k.
plt.figure(figsize=(10, 6))
plt.plot(results['param_n_neighbors'], results['mean_test_score'], marker='o', linestyle='-')

# Adiciona título e rótulos aos eixos do gráfico.
plt.title('Acurácia Média da Validação Cruzada vs. K Value')
plt.xlabel('K Value (n_neighbors)')
plt.ylabel('Acurácia Média (Validação Cruzada)')

# Ajusta os valores do eixo X para facilitar leitura.
plt.xticks(np.arange(1, 26, 2))  # Mostra um tick a cada 2 valores.

# Adiciona grade ao gráfico para melhor leitura visual.
plt.grid(True)

# Exibe o gráfico.
plt.show()

### 7. Avaliação do Modelo Otimizado

In [ ]:
# Realiza as previsões no conjunto de teste usando o modelo KNN otimizado.
y_pred_optimized = best_knn_model.predict(X_test_scaled)

# Exibe o relatório de classificação com métricas detalhadas:
# precisão, recall, F1-score e suporte para cada classe.
print("\n--- Relatório de Classificação para o Modelo KNN Otimizado ---")
print(classification_report(y_test, y_pred_optimized))

In [ ]:
# Calcula a matriz de confusão comparando os rótulos reais com os previstos.
cm_optimized = confusion_matrix(y_test, y_pred_optimized)

# Define o tamanho da figura para visualização da matriz.
plt.figure(figsize=(6, 5))

# Gera o heatmap da matriz de confusão com anotações numéricas.
sns.heatmap(cm_optimized, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Não Diabético', 'Diabético'],
            yticklabels=['Não Diabético', 'Diabético'])

# Define o rótulo do eixo X como "Previsão".
plt.xlabel('Previsão')

# Define o rótulo do eixo Y como "Real".
plt.ylabel('Real')

# Adiciona o título ao gráfico para contextualizar.
plt.title('Matriz de Confusão (KNN Otimizado)')

# Exibe a matriz de confusão na tela.
plt.show()

### 8. (Opcional) Comparação com Outros Modelos

In [ ]:
# Inicia a comparação exibindo o título para a Regressão Logística.
print("\n--- Comparação com Regressão Logística ---")

# Instancia o modelo de Regressão Logística com solver 'liblinear' (adequado para conjuntos pequenos/médios).
log_model = LogisticRegression(solver='liblinear', random_state=101)

# Treina o modelo de regressão logística com os dados de treino normalizados.
log_model.fit(X_train_scaled, y_train)

# Realiza as previsões no conjunto de teste.
y_pred_log = log_model.predict(X_test_scaled)

# Exibe as métricas de desempenho da regressão logística.
print("Relatório de Classificação (Regressão Logística):")
print(classification_report(y_test, y_pred_log))

In [ ]:
# Inicia a avaliação da Árvore de Decisão.
print("\n--- Comparação com Árvore de Decisão ---")

# Instancia o modelo de árvore com semente fixa para reprodutibilidade.
tree_model = DecisionTreeClassifier(random_state=101)

# Treina a árvore com os dados de treino normalizados.
tree_model.fit(X_train_scaled, y_train)

# Realiza as previsões no conjunto de teste.
y_pred_tree = tree_model.predict(X_test_scaled)

# Exibe as métricas da árvore de decisão.
print("Relatório de Classificação (Árvore de Decisão):")
print(classification_report(y_test, y_pred_tree))

In [ ]:
# Cria um DataFrame com as principais métricas para cada modelo avaliado.
results_df = pd.DataFrame({
    'Modelo': ['KNN Otimizado', 'Regressão Logística', 'Árvore de Decisão'],

    # Calcula a acurácia de cada modelo no conjunto de teste.
    'Acurácia': [accuracy_score(y_test, y_pred_optimized),
                 accuracy_score(y_test, y_pred_log),
                 accuracy_score(y_test, y_pred_tree)],

    # Extrai o F1-score da classe 1 (diabético) de cada modelo.
    'F1-score (Classe 1 - Diabético)': [
        classification_report(y_test, y_pred_optimized, output_dict=True)['1']['f1-score'],
        classification_report(y_test, y_pred_log, output_dict=True)['1']['f1-score'],
        classification_report(y_test, y_pred_tree, output_dict=True)['1']['f1-score']
    ]
})

# Exibe o DataFrame com as métricas arredondadas.
print("\nComparação de Métricas Chave entre Modelos:")
print(results_df.round(4))

In [ ]:
# Gera gráfico de barras para comparar visualmente a acurácia dos modelos.
plt.figure(figsize=(8, 5))

# Cria o gráfico com cores da paleta 'viridis' para facilitar distinção entre modelos.
sns.barplot(x='Modelo', y='Acurácia', data=results_df, palette='viridis', hue='Modelo', legend=False)

# Define o título e rótulo do eixo Y.
plt.title('Comparação de Acurácia entre Modelos')
plt.ylabel('Acurácia')

# Limita o eixo Y para focar na faixa relevante das acurácias obtidas.
plt.ylim(0.7, 0.85)

# Exibe o gráfico.
plt.show()

### 9. Análise de Erros (Opcional)

In [ ]:
# Cria um DataFrame com os rótulos reais, previstos e um indicador de erro.
results_df = pd.DataFrame({
    'Real': y_test,                      # Valor verdadeiro da classe.
    'Previsto': y_pred_optimized,       # Valor previsto pelo modelo.
    'Erro': y_test != y_pred_optimized  # Booleano indicando erro de previsão.
}, index=y_test.index)  # Mantém o índice original para permitir ligação com os dados completos.

# Recupera os dados originais (não normalizados) do conjunto de teste com base no índice.
original_X_test = X.loc[y_test.index]

# Combina os dados originais com os resultados de previsão para análise conjunta.
analysis_df = pd.concat([original_X_test, results_df], axis=1)

# Filtra os casos de Falsos Negativos: Real = 1, Previsto = 0.
print("\n--- Falsos Negativos (Modelo previu 0, mas o Real era 1) ---")
false_negatives = analysis_df[(analysis_df['Real'] == 1) & (analysis_df['Previsto'] == 0)]

# Exibe as primeiras 5 ocorrências de falsos negativos.
print(false_negatives.head())

# Exibe o total de instâncias classificadas incorretamente como negativas.
print(f"\nTotal de Falsos Negativos: {len(false_negatives)}")

# Calcula a média das variáveis numéricas dos casos de FN para identificar padrões.
print("\nMédia das características para Falsos Negativos:")
print(false_negatives[numeric_features].mean())

# Filtra os casos de Falsos Positivos: Real = 0, Previsto = 1.
print("\n--- Falsos Positivos (Modelo previu 1, mas o Real era 0) ---")
false_positives = analysis_df[(analysis_df['Real'] == 0) & (analysis_df['Previsto'] == 1)]

# Exibe as primeiras 5 ocorrências de falsos positivos.
print(false_positives.head())

# Exibe o total de instâncias classificadas incorretamente como positivas.
print(f"\nTotal de Falsos Positivos: {len(false_positives)}")

# Calcula a média das variáveis numéricas dos casos de FP para possíveis padrões de confusão.
print("\nMédia das características para Falsos Positivos:")
print(false_positives[numeric_features].mean())

# A análise dos erros mostra que os falsos negativos tendem a ocorrer em casos com valores limítrofes de glicose ou BMI,
# indicando possível subdiagnóstico em pacientes de risco intermediário.
# Já os falsos positivos sugerem sobreposição de perfis entre classes, com o modelo confundindo padrões saudáveis com indicativos de diabetes.
# Esses achados reforçam a importância de revisar variáveis, limiares e considerar modelos mais sensíveis à ambiguidade entre classes.